In [1]:
import gzip
import glob
import sys
import os
import shutil
import struct
import pickle
import numpy as np
import matplotlib.pyplot as plt

os.chdir('/Users/hiroyukiinoue/Projects/Python/bayes_nn')

In [2]:
pwd

'/Users/hiroyukiinoue/Projects/Python/bayes_nn'

In [3]:
if not os.path.exists('./data/raw'):
    os.mkdir('./data/raw')

files = glob.glob('./data/raw/*')
for file in files:
    os.remove(file)

In [4]:
!curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!mv ./t*-ubyte.gz ./data/raw
!gunzip ./data/raw/t*-ubyte.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28881  100 28881    0     0  52861      0 --:--:-- --:--:-- --:--:-- 52895
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9680k  100 9680k    0     0   438k      0  0:00:22  0:00:22 --:--:--  696k 0     0   245k      0  0:00:39  0:00:07  0:00:32  319k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4542  100  4542    0     0  11373      0 --:--:-- --:--:-- --:--:-- 11355
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1610k  100 1610k    0     0   567k      0  0:00:02  0:00:02 --:--:--  56

# refer to http://yann.lecun.com/exdb/mnist/

In [5]:
'''FILE FORMATS FOR THE MNIST DATABASE
The data is stored in a very simple file format designed for storing vectors and multidimensional matrices. General info on this format is given at the end of this page, but you don't need to read that to use the data files.
All the integers in the files are stored in the MSB first (high endian) format used by most non-Intel processors. Users of Intel processors and other low-endian machines must flip the bytes of the header.

There are 4 files:

train-images-idx3-ubyte: training set images
train-labels-idx1-ubyte: training set labels
t10k-images-idx3-ubyte:  test set images
t10k-labels-idx1-ubyte:  test set labels

The training set contains 60000 examples, and the test set 10000 examples.

The first 5000 examples of the test set are taken from the original NIST training set. The last 5000 are taken from the original NIST test set. The first 5000 are cleaner and easier than the last 5000.
'''

"FILE FORMATS FOR THE MNIST DATABASE\nThe data is stored in a very simple file format designed for storing vectors and multidimensional matrices. General info on this format is given at the end of this page, but you don't need to read that to use the data files.\nAll the integers in the files are stored in the MSB first (high endian) format used by most non-Intel processors. Users of Intel processors and other low-endian machines must flip the bytes of the header.\n\nThere are 4 files:\n\ntrain-images-idx3-ubyte: training set images\ntrain-labels-idx1-ubyte: training set labels\nt10k-images-idx3-ubyte:  test set images\nt10k-labels-idx1-ubyte:  test set labels\n\nThe training set contains 60000 examples, and the test set 10000 examples.\n\nThe first 5000 examples of the test set are taken from the original NIST training set. The last 5000 are taken from the original NIST test set. The first 5000 are cleaner and easier than the last 5000.\n"

In [6]:
'''
[offset] [type]          [value]          [description]
0000     32 bit integer  0x00000801(2049) magic number (MSB first)
0004     32 bit integer  60000            number of items
0008     unsigned byte   ??               label
0009     unsigned byte   ??               label
........
xxxx     unsigned byte   ??               label
The labels values are 0 to 9.
'''

with open('./data/raw/train-labels-idx1-ubyte', 'rb') as f:
    zero, data_type, dims = struct.unpack('>HBB', f.read(4))
    shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
    data = np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
    with open('./data/raw/train-labels-idx1-ubyte.pkl','wb') as g:
        pickle.dump(data, g)

print(zero, data_type, dims)
print(shape)
print(data.shape)

0 8 1
(60000,)
(60000,)


/anaconda/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app


In [7]:
'''
[offset] [type]          [value]          [description]
0000     32 bit integer  0x00000803(2051) magic number
0004     32 bit integer  60000            number of images
0008     32 bit integer  28               number of rows
0012     32 bit integer  28               number of columns
0016     unsigned byte   ??               pixel
0017     unsigned byte   ??               pixel
........
xxxx     unsigned byte   ??               pixel
Pixels are organized row-wise. Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black).
'''

with open('./data/raw/train-images-idx3-ubyte', 'rb') as f:
    zero, data_type, dims = struct.unpack('>HBB', f.read(4))
    shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
    data = np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
    with open('./data/raw/train-images-idx3-ubyte.pkl','wb') as g:
        pickle.dump(data, g)
    
print(zero, data_type, dims)
print(shape)
print(data.shape)

/anaconda/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


0 8 3
(60000, 28, 28)
(60000, 28, 28)


In [8]:
'''
[offset] [type]          [value]          [description]
0000     32 bit integer  0x00000801(2049) magic number (MSB first)
0004     32 bit integer  10000            number of items
0008     unsigned byte   ??               label
0009     unsigned byte   ??               label
........
xxxx     unsigned byte   ??               label
The labels values are 0 to 9.
'''

with open('./data/raw/t10k-labels-idx1-ubyte', 'rb') as f:
    zero, data_type, dims = struct.unpack('>HBB', f.read(4))
    shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
    data = np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
    with open('./data/raw/t10k-labels-idx1-ubyte.pkl','wb') as g:
        pickle.dump(data, g)
    
print(zero, data_type, dims)
print(shape)
print(data.shape)

0 8 1
(10000,)
(10000,)


/anaconda/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app


In [9]:
'''
[offset] [type]          [value]          [description]
0000     32 bit integer  0x00000803(2051) magic number
0004     32 bit integer  10000            number of images
0008     32 bit integer  28               number of rows
0012     32 bit integer  28               number of columns
0016     unsigned byte   ??               pixel
0017     unsigned byte   ??               pixel
........
xxxx     unsigned byte   ??               pixel
Pixels are organized row-wise. Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black).
'''

with open('./data/raw/t10k-images-idx3-ubyte', 'rb') as f:
    zero, data_type, dims = struct.unpack('>HBB', f.read(4))
    shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
    data = np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
    with open('./data/raw/t10k-images-idx3-ubyte.pkl','wb') as g:
        pickle.dump(data, g)

print(zero, data_type, dims)
print(shape)
print(data.shape)

0 8 3
(10000, 28, 28)
(10000, 28, 28)


/anaconda/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [10]:
!rm ./data/raw/t*-ubyte.gz
!rm ./data/raw/t*-ubyte

rm: ./data/raw/t*-ubyte.gz: No such file or directory
